In [7]:
# | default_exp agent.planner

In [2]:
# | export
import json
from typing import Dict, List, Any, Optional, Union
from datetime import datetime
from pathlib import Path
from enum import Enum

from pydantic import BaseModel, Field, validator
from rich.console import Console

from agentic.core.agent import Agent, AgentConfig
from agentic.agent.debater import create_debate
from agentic.tools.introspect import IntrospectTool
from agentic.tools.base import ToolMetadata, ToolCategory

class ExecutionStatus(str, Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    SUCCESS = "success"
    FAILED = "failed"
    RETRYING = "retrying"

class ExecutionMode(Enum):
    SEQUENTIAL = "sequential"
    PARALLEL = "parallel"

class Resources(BaseModel):
    previous: List[str] = Field(default_factory=list, description="Resources created in previous steps")
    required: List[str] = Field(default_factory=list, description="Resources helpful or required for this/current next step")
    
class ActionStep(BaseModel):
    step: int = Field(..., description="The step number")
    action: str = Field(..., description="The action description")
    tool: Optional[str] = Field(None, description="The tool to use (from available tools)")
    purpose: str = Field(..., description="The purpose of this step")
    sub_steps: List[str] = Field(default_factory=list, description="Sub-steps if any")
    introspect_after: bool = Field(True, description="Whether to call introspect after this step")
    system_prompt: str = Field(..., description="Self-sufficient system prompt for this subtask")
    user_prompt: str = Field(..., description="Self-sufficient user prompt for this subtask")
    introspect_prompt: str = Field(..., description="Template prompt for introspect")
    resources: Resources = Field(default_factory=Resources, description="Resources for this subtask")
    execution_mode: ExecutionMode = Field(ExecutionMode.SEQUENTIAL, description="Sequential or parallel execution")
    
class Task(BaseModel):
    id: str = Field(..., description="Unique task ID, e.g., T001")
    name: str = Field(..., description="Concise task name")
    description: str = Field(..., description="Detailed task description")
    dependencies: List[str] = Field(default_factory=list, description="List of dependent task IDs")
    tools: List[str] = Field(default_factory=list, description="List of tool names")
    actions: List[ActionStep] = Field(default_factory=list, description="List of action steps")
    success_criteria: str = Field(..., description="Criteria for task success")
    expected_outputs: List[str] = Field(default_factory=list, description="Expected output artifacts")
    resources: Resources = Field(default_factory=Resources, description="Overall resources for the task")
    potential_options: List[str] = Field(default_factory=list, description="List of potential options for decision making")
    needs_debate: bool = Field(False, description="Whether this task requires debating options")

    
class ActionResult(BaseModel):
    action_id: str = Field(..., description="Unique action identifier")
    tool_used: str = Field(..., description="Tool that was executed")
    result: Any = Field(..., description="Raw result from tool execution")
    execution_time: float = Field(..., description="Time taken in seconds")
    status: ExecutionStatus = Field(..., description="Execution status")
    error_message: Optional[str] = Field(None, description="Error message if failed")
    artifacts_created: List[str] = Field(default_factory=list, description="Files/artifacts created")


class IntrospectionResult(BaseModel):
    success: bool = Field(..., description="Whether introspection passed")
    score: float = Field(..., description="Performance score 0-10")
    feedback: str = Field(..., description="Detailed feedback")
    next_action: str = Field(..., description="proceed, retry, or escalate")
    recommendations: List[str] = Field(default_factory=list, description="Improvement suggestions")


class TaskExecutionResult(BaseModel):
    task_id: str = Field(..., description="Task identifier")
    task_name: str = Field(..., description="Task name")
    status: ExecutionStatus = Field(..., description="Overall task status")
    actions_executed: List[ActionResult] = Field(default_factory=list, description="ActionStep results")
    introspection_results: List[IntrospectionResult] = Field(default_factory=list, description="Introspection results")
    artifacts_created: List[str] = Field(default_factory=list, description="All artifacts from this task")
    execution_time: float = Field(..., description="Total task execution time")
    timestamp: datetime = Field(default_factory=datetime.now, description="Execution timestamp")
    success_criteria_met: bool = Field(..., description="Whether success criteria were met")


class ProjectContext(BaseModel):
    original_request: str = Field(..., description="Original user request")
    execution_history: List[TaskExecutionResult] = Field(default_factory=list, description="Task execution history")
    current_artifacts: List[str] = Field(default_factory=list, description="All project artifacts")
    total_tasks_completed: int = Field(0, description="Number of completed tasks")
    project_status: ExecutionStatus = Field(ExecutionStatus.PENDING, description="Overall project status")


class ProjectBreakdown(BaseModel):
    project_summary: str = Field(..., description="High-level project summary")
    phases: List[str] = Field(..., description="Major project phases")
    key_deliverables: List[str] = Field(..., description="Key deliverables expected")
    technical_approach: str = Field(..., description="Technical approach and architecture")

class DynamicTaskExecutor:
    """Production-ready dynamic task execution system"""
    
    def __init__(self, model: str = "gpt-oss:20b"):
        self.model = model
        # Create AgentConfig with standard settings
        config = AgentConfig(
            name="dynamic_executor",
            instructions="You are an AI agent that executes tasks systematically. Analyze requirements, break down complex problems, and provide detailed technical responses with proper implementation.",
            model=model,
            tools=["fs_read", "fs_write", "execute_bash", "code_interpreter"],
            temperature=0.7
        )
        self.agent = Agent(config)
        self.introspect = IntrospectTool(
            metadata=ToolMetadata(
                name="introspect",
                description="Task validation and feedback tool",
                category=ToolCategory.INTELLIGENCE
            )
        )
        self.console = Console()
        
        # Execution configuration
        self.max_retries = 3
        self.max_tasks = 20
        # State
        self.context: Optional[ProjectContext] = None
        self.project_breakdown: Optional[ProjectBreakdown] = None
        self.estimated_total_tasks: int = 0  # To store estimated total tasks
    
    def execute_project(self, user_request: str) -> ProjectContext:
        """Main execution loop with two-phase approach"""
        self.console.print(f"🚀 Starting dynamic execution for: {user_request}")
        
        # Phase 1: Generate project breakdown
        self.console.print("\n📋 PHASE 1: Generating project breakdown...")
        self.project_breakdown = self._generate_project_breakdown(user_request)
        
        if not self.project_breakdown:
            self.console.print("❌ Failed to generate project breakdown")
            return ProjectContext(original_request=user_request, project_status=ExecutionStatus.FAILED)
        
        self.console.print(f"✅ Project breakdown complete")
        
        # Initialize project context
        self.context = ProjectContext(original_request=user_request)
        
        # Estimate total tasks from breakdown
        self.estimated_total_tasks = self._estimate_total_tasks()
        
        # Phase 2: Execute tasks one by one
        self.console.print("\n🔄 PHASE 2: Executing tasks dynamically...")
        
        task_counter = 1
        
        while not self._is_project_complete() and task_counter <= self.max_tasks:
            self.console.print(f"\n{'='*60}")
            self.console.print(f"📋 GENERATING TASK {task_counter}")
            self.console.print(f"{'='*60}")
            
            # Generate next task using breakdown context
            task = self._generate_next_task_with_breakdown()
            if not task:
                self.console.print("❌ No more tasks to generate. Project complete.")
                break
            
            self.console.print(f"✅ Generated: {task.name}")
            
            # Execute task
            task_result = self._execute_task(task)
            
            # Update context
            self.context.execution_history.append(task_result)
            self.context.current_artifacts.extend(task_result.artifacts_created)
            self.context.total_tasks_completed += 1
            
            # Update project status
            if task_result.status == ExecutionStatus.FAILED:
                self.context.project_status = ExecutionStatus.FAILED
                self.console.print(f"❌ Task {task.id} failed. Stopping execution.")
                break
            
            task_counter += 1
        
        # Final project status
        if self.context.project_status != ExecutionStatus.FAILED:
            self.context.project_status = ExecutionStatus.SUCCESS
        
        self.console.print(f"\n🎉 Project execution completed! Generated {len(self.context.execution_history)} tasks.")
        return self.context
    
    def _generate_project_breakdown(self, user_request: str) -> Optional[ProjectBreakdown]:
        """Phase 1: Generate structured project breakdown with task-wise organization"""
        
        prompt = f"""
Analyze this project request and provide a structured breakdown for AI agent execution:

REQUEST: {user_request}

Provide response in this EXACT format:

PROJECT SUMMARY:
[One paragraph overview of what needs to be built]

MAJOR PHASES:
Phase 1: [Phase name] - [Brief description]
Phase 2: [Phase name] - [Brief description]
Phase 3: [Phase name] - [Brief description]
[Continue as needed]

PHASE TASKS:
Phase 1 Tasks:
- Task 1.1: Analyze requirements and create specifications
- Task 1.2: Setup project structure and configuration
- Task 1.3: Initialize development environment

Phase 2 Tasks:
- Task 2.1: Implement core functionality
- Task 2.2: Create data models and interfaces
- Task 2.3: Build processing logic

[Continue for all phases with AI-executable tasks]

KEY DELIVERABLES:
- [Specific file/component 1]
- [Specific file/component 2]
- [Continue list]

TECHNICAL APPROACH:
[Architecture and technology decisions in 2-3 sentences]

Use AI automation terminology: analyze, generate, implement, validate, deploy.
Focus on tasks that can be executed programmatically by AI agents.
Avoid human-centric activities like interviews, meetings, or manual processes.
"""
        
        try:
            response = self.agent.run(prompt).get("content", "")
            
            # Save breakdown for reference
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            with open(f"project_breakdown_{timestamp}.txt", 'w') as f:
                f.write(response)
            
            # Parse the response into structured format
            breakdown = self._parse_breakdown_response(response)
            return breakdown
            
        except Exception as e:
            self.console.print(f"❌ Project breakdown generation failed: {e}")
            return None
    
    def _parse_breakdown_response(self, response: str) -> ProjectBreakdown:
        """Parse breakdown response into structured format"""
        
        # Simple parsing - extract key information
        lines = response.split('\n')
        
        project_summary = "Project breakdown generated"
        phases = []
        deliverables = []
        technical_approach = "Standard approach"
        
        current_section = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Detect sections
            if "PROJECT SUMMARY" in line.upper() or "SUMMARY" in line.upper():
                current_section = "summary"
            elif "PHASES" in line.upper() or "PHASE" in line.upper():
                current_section = "phases"
            elif "DELIVERABLES" in line.upper():
                current_section = "deliverables"
            elif "TECHNICAL" in line.upper():
                current_section = "technical"
            elif line.startswith('-') or line.startswith('•') or line.startswith('*'):
                # List item
                item = line.lstrip('-•* ').strip()
                if current_section == "phases":
                    phases.append(item)
                elif current_section == "deliverables":
                    deliverables.append(item)
            elif current_section == "summary" and len(line) > 20:
                project_summary = line
            elif current_section == "technical" and len(line) > 20:
                technical_approach = line
        
        return ProjectBreakdown(
            project_summary=project_summary,
            phases=phases if phases else ["Setup", "Development", "Testing", "Deployment"],
            key_deliverables=deliverables if deliverables else ["Core components", "Documentation", "Tests"],
            technical_approach=technical_approach
        )
    
    def _estimate_total_tasks(self) -> int:
        """Estimate total number of tasks based on project breakdown"""
        # Count tasks listed in the breakdown phases
        task_count = 0
        breakdown_text = "\n".join(self.project_breakdown.phases)
        for line in breakdown_text.split('\n'):
            if line.strip().startswith('-') or line.strip().startswith('•') or line.strip().startswith('*'):
                task_count += 1
        # Add buffer for additional tasks (e.g., validation, deployment)
        return max(task_count, 5)  # Ensure at least 5 tasks for small projects
    
    def _generate_next_task_with_breakdown(self) -> Optional[Task]:
        """Generate single next task using project breakdown context"""
        
        context_prompt = self._build_context_prompt()
        
        prompt = f"""
CRITICAL: You MUST generate EXACTLY ONE task object (not an array). Generate the NEXT task that follows the previous ones without repeating any previous tasks. If no further tasks are needed, return null.

PROJECT BREAKDOWN CONTEXT:
Summary: {self.project_breakdown.project_summary}
Phases: {', '.join(self.project_breakdown.phases)}
Key Deliverables: {', '.join(self.project_breakdown.key_deliverables)}
Technical Approach: {self.project_breakdown.technical_approach}

ORIGINAL REQUEST: {self.context.original_request}

{context_prompt}

Generate the NEXT SINGLE TASK based on the project breakdown and execution history. Ensure the task is unique and not a repeat of any previous tasks listed in the execution history.

If the task involves decision making (e.g., choosing frameworks, libraries, architectures, or any other options), set "needs_debate": true and provide "potential_options": ["option1", "option2", ...] with all reasonable potential options. This will trigger the debate agent to decide the best one. Do this not only for requirements but for any task requiring decision making.

IMPORTANT: Ensure EVERY action in the "actions" list has ALL the following fields completely filled out: 
- step: integer
- action: string (e.g., "analyze", "implement", "introspect")
- tool: string or null (e.g., "fs_write" if using a tool)
- purpose: string
- sub_steps: array of strings
- introspect_after: boolean
- system_prompt: string (must be a detailed, self-sufficient system prompt)
- user_prompt: string (must be a detailed, self-sufficient user prompt)
- introspect_prompt: string (must be a template for validation)
- resources: object with "previous" and "required" arrays
- execution_mode: "sequential" or "parallel"

Do NOT omit any fields. Fill them appropriately even if some are empty arrays or default values.

RESPOND WITH SINGLE TASK OBJECT (NOT ARRAY) OR NULL:
{{
  "id": "T{len(self.context.execution_history) + 1:03d}",
  "name": "Next Task Name",
  "description": "Task description aligned with project phases",
  "dependencies": [],
  "tools": [],
  "actions": [
    {{
      "step": 1,
      "action": "analyze",
      "tool": null,
      "purpose": "Analyze requirements",
      "sub_steps": ["Review breakdown", "Identify needs"],
      "introspect_after": true,
      "system_prompt": "You are an analyzer. CONTEXT: {self.project_breakdown.project_summary}. Analyze the following...",
      "user_prompt": "Based on project breakdown, analyze: specific instructions...",
      "introspect_prompt": "Validate analysis by checking if it covers all aspects...",
      "resources": {{"previous": [], "required": ["project_breakdown.txt"]}},
      "execution_mode": "sequential"
    }},
    ...
    {{
      "step": n,
      "action": "implement",
      "tool": "code_interpreter",
      "purpose": "Implement component",
      "sub_steps": ["Write code", "Test locally"],
      "introspect_after": true,
      "system_prompt": "You are a coder. CONTEXT: {self.project_breakdown.project_summary}. Implement the following...",
      "user_prompt": "Implement the component as per analysis...",
      "introspect_prompt": "Validate implementation by checking functionality...",
      "resources": {{"previous": ["analysis.txt"], "required": []}},
      "execution_mode": "sequential"
    }}
  ],
  "success_criteria": "Specific success criteria",
  "expected_outputs": ["output1.py"],
  "resources": {{"previous": [], "required": []}},
  "potential_options": [],
  "needs_debate": false
}}

OR:
null

IMPORTANT: Return ONLY the task object or null, NOT an array of tasks. Ensure the task ID is unique and follows the format TXXX (e.g., T001, T002).
"""
        
        try:
            response = self.agent.run(prompt).get("content", "")
            
            # Save raw response for debugging
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            with open(f"task_generation_raw_{timestamp}.txt", 'w') as f:
                f.write(response)
            
            # Parse and validate task
            response = response.strip()
            if response.lower() == "null":
                return None
            
            task_data = json.loads(response)
            
            # If LLM returns array, take only the first task
            if isinstance(task_data, list):
                if len(task_data) > 0:
                    task_data = task_data[0]
                    self.console.print("⚠️ LLM returned multiple tasks, using only the first one")
                else:
                    raise ValueError("Empty task array returned")
            
            task = Task(**task_data)
            task.id = f"T{len(self.context.execution_history) + 1:03d}"  # Enforce correct task ID
            
            # Check for task repetition
            for prev_task in self.context.execution_history:
                if task.name.lower() == prev_task.task_name.lower():
                    self.console.print(f"⚠️ Task {task.name} appears to be a repeat. Skipping.")
                    return None
            
            return task
            
        except Exception as e:
            self.console.print(f"❌ Task generation failed: {e}")
            return None
    
    def _execute_task(self, task: Task) -> TaskExecutionResult:
        """Execute all actions in a task with full tracking"""
        start_time = datetime.now()
        self.console.print(f"\n🔄 Executing Task: {task.name}")
        
        if task.needs_debate:
            self.console.print("🗣️ Initiating debate for decision making...")
            # Assuming create_debate takes topic and options; adjust based on actual signature
            debate_result = create_debate(topic=task.description, options=task.potential_options)
            # TODO: Integrate debate_result into task execution, e.g., update task.description or resources
            self.console.print(f"Debate result: {debate_result}")
        
        task_result = TaskExecutionResult(
            task_id=task.id,
            task_name=task.name,
            status=ExecutionStatus.IN_PROGRESS,
            execution_time=0.0,
            success_criteria_met=False
        )
        
        overall_success = True
        
        for action in task.actions:
            self.console.print(f"\n  📌 Step {action.step}: {action.purpose}")
            
            # Execute action with retries
            action_result, introspection_result = self._execute_action_with_retries(task, action)
            
            # Store results
            task_result.actions_executed.append(action_result)
            if introspection_result:
                task_result.introspection_results.append(introspection_result)
            
            # Update artifacts
            task_result.artifacts_created.extend(action_result.artifacts_created)
            
            if action_result.status != ExecutionStatus.SUCCESS:
                self.console.print(f"  ❌ Step {action.step} failed after {self.max_retries} retries")
                overall_success = False
                break
            else:
                self.console.print(f"  ✅ Step {action.step} completed successfully")
        
        # Finalize task result
        end_time = datetime.now()
        task_result.execution_time = (end_time - start_time).total_seconds()
        task_result.status = ExecutionStatus.SUCCESS if overall_success else ExecutionStatus.FAILED
        task_result.success_criteria_met = overall_success
        
        return task_result
    
    def _execute_action_with_retries(self, task: Task, action: ActionStep) -> tuple[ActionResult, Optional[IntrospectionResult]]:
        """Execute single action with introspection and retries"""
        
        action_result = ActionResult(
            action_id=f"{task.id}_step_{action.step}",
            tool_used=action.action,
            result="",
            execution_time=0.0,
            status=ExecutionStatus.PENDING
        )
        
        introspection_result = None
        
        for attempt in range(self.max_retries):
            self.console.print(f"    🔄 Attempt {attempt + 1}/{self.max_retries}")
            
            try:
                start_time = datetime.now()
                
                # Execute the action
                result = self._execute_single_action(action)
                
                end_time = datetime.now()
                execution_time = (end_time - start_time).total_seconds()
                
                # Update action result
                action_result.result = result
                action_result.execution_time = execution_time
                action_result.status = ExecutionStatus.SUCCESS
                action_result.artifacts_created = self._detect_new_artifacts()
                
                self.console.print(f"    📝 ActionStep completed in {execution_time:.2f}s")
                
                # Introspect if required
                if action.introspect_after:
                    introspection_result = self._introspect_action(task, action, result)
                    
                    if introspection_result.success:
                        self.console.print(f"    ✅ Introspection passed (score: {introspection_result.score})")
                        return action_result, introspection_result
                    else:
                        self.console.print(f"    ⚠️ Introspection failed (score: {introspection_result.score})")
                        if attempt < self.max_retries - 1:
                            action_result.status = ExecutionStatus.RETRYING
                            self.console.print(f"    🔄 Retrying with feedback: {introspection_result.feedback}")
                            continue
                else:
                    return action_result, None
                    
            except Exception as e:
                self.console.print(f"    ❌ ActionStep execution error: {e}")
                action_result.error_message = str(e)
                action_result.status = ExecutionStatus.FAILED
                
                if attempt < self.max_retries - 1:
                    action_result.status = ExecutionStatus.RETRYING
                    continue
        
        # All retries exhausted
        action_result.status = ExecutionStatus.FAILED
        return action_result, introspection_result
    
    def _execute_single_action(self, action: ActionStep) -> str:
        """Execute single action using Agent's tool system"""
        
        # Build enriched prompt with full context
        enriched_prompt = f"""
SYSTEM CONTEXT:
{action.system_prompt}

USER REQUEST:
{action.user_prompt}

EXECUTION DETAILS:
- Tool: {action.action}
- Purpose: {action.purpose}
- Sub-steps: {', '.join(action.sub_steps)}
- Execution Mode: {action.execution_mode}

RESOURCES:
- Previous: {', '.join(action.resources.previous)}
- Required: {', '.join(action.resources.required)}

Execute this action and provide detailed output.
"""
        
        # Call agent with the enriched prompt
        result = self.agent.run(enriched_prompt).get("content", "")
        
        # TODO: Integrate with actual tool execution based on action.action
        # For now, return LLM response
        return result
    
    def _introspect_action(self, task: Task, action: ActionStep, result: str) -> IntrospectionResult:
        """Use introspect tool to validate action success"""
        
        try:
            # Call introspect tool
            introspect_result = self.introspect.execute(
                action="task_review",
                task_review={
                    "task_id": task.id,
                    "task_description": f"{task.description} - Step {action.step}: {action.purpose}",
                    "solution_provided": result,
                    "success_criteria": [task.success_criteria],
                    "actual_result": result,
                    "execution_time": 1.0,
                    "retry_count": 0
                }
            )
            
            if introspect_result and 'performance_score' in introspect_result:
                score = introspect_result['performance_score']
                success = introspect_result.get('success', False)  # Use introspect tool's success determination
                feedback = introspect_result.get('feedback_for_retry', 'No feedback provided')
                next_action = introspect_result.get('next_action', 'proceed' if success else 'retry')
                recommendations = introspect_result.get('recommendations', [])
                
                return IntrospectionResult(
                    success=success,
                    score=score,
                    feedback=feedback,
                    next_action=next_action,
                    recommendations=recommendations
                )
            
        except Exception as e:
            self.console.print(f"⚠️ Introspection error: {e}")
        
        return IntrospectionResult(
            success=False,
            score=0.0,
            feedback="Introspection failed",
            next_action="retry",
            recommendations=["Fix introspection system"]
        )
    
    def _build_context_prompt(self) -> str:
        """Build comprehensive context from execution history"""
        if not self.context.execution_history:
            return "PREVIOUS CONTEXT: This is the first task."
        
        context_parts = ["EXECUTION HISTORY:"]
        
        # Include all tasks for full context to avoid repetition
        for i, task_result in enumerate(self.context.execution_history, 1):
            status_emoji = "✅" if task_result.status == ExecutionStatus.SUCCESS else "❌"
            context_parts.append(f"{i}. {task_result.task_name} (ID: {task_result.task_id}) - {status_emoji} {task_result.status}")
            context_parts.append(f"   Description: {task_result.task_name}")
            if task_result.artifacts_created:
                context_parts.append(f"   Artifacts: {', '.join(task_result.artifacts_created[:3])}")
        
        context_parts.append(f"\nTOTAL ARTIFACTS: {len(self.context.current_artifacts)}")
        context_parts.append(f"PROJECT STATUS: {self.context.project_status}")
        context_parts.append(f"PROGRESS: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks completed")
        
        return "\n".join(context_parts)
    
    def _detect_new_artifacts(self) -> List[str]:
        """Detect newly created files/artifacts"""
        current_files = set()
        current_dir = Path(".")
        
        for file_path in current_dir.glob("**/*"):
            if file_path.is_file() and not file_path.name.startswith('.'):
                current_files.add(str(file_path))
        
        previous_files = set(self.context.current_artifacts)
        new_files = current_files - previous_files
        
        return list(new_files)
    
    def _is_project_complete(self) -> bool:
        """Determine if project is complete based on task generation and progress"""
        if not self.context.execution_history:
            return False
        
        # Check if no further tasks are generated
        next_task = self._generate_next_task_with_breakdown()
        if next_task is None:
            self.console.print(f"✅ No further tasks needed. Project completion: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks")
            return True
        
        # Calculate completion percentage
        completion_percentage = (self.context.total_tasks_completed / self.estimated_total_tasks) * 100
        self.console.print(f"📊 Project progress: {self.context.total_tasks_completed}/{self.estimated_total_tasks} tasks ({completion_percentage:.1f}%)")
        
        return False
    
    def save_execution_report(self, filename: str = None) -> str:
        """Save detailed execution report"""
        if not filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"execution_report_{timestamp}.json"
        
        with open(filename, 'w') as f:
            json.dump(self.context.model_dump(), f, indent=2, default=str)
        
        self.console.print(f"📊 Execution report saved to: {filename}")
        return filename



def main():
    """Production demo with proper error handling"""
    try:
        executor = DynamicTaskExecutor("gpt-oss:20b")
        
        user_request = "Build a web scraping framework with rate limiting and data storage"
        
        # Execute project
        result = executor.execute_project(user_request)
        
        # Save report
        # report_file = executor.save_execution_report()
        
        # print(f"\n📈 EXECUTION SUMMARY:")
        # print(f"   Total Tasks: {result.total_tasks_completed}")
        # print(f"   Project Status: {result.project_status}")
        # print(f"   Artifacts Created: {len(result.current_artifacts)}")
        # print(f"   Report: {report_file}")
        
    except KeyboardInterrupt:
        print("\n⚠️ Execution interrupted by user.")
    except Exception as e:
        print(f"❌ Execution failed: {e}")


main()

2025-09-26 19:48:10,993 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 19:48:10,994 - INFO - Initialized LLM client with model: gpt-oss:20b


🚀 Starting dynamic execution for: Build a web scraping framework with rate limiting and data storage

📋 PHASE 1: Generating project breakdown...

2025-09-26 19:48:13,959 - INFO - HTTP Request: POST http://192.168.29.147:11500/v1/chat/completions "HTTP/1.1 200 OK"


We need to produce a structured breakdown for AI agent execution in the exact given format.

...

📝 ActionStep completed in 105.71s

⚠️ Introspection failed (score: 0.0)

❌ Step 1 failed after 3 retries

❌ Task T001 failed. Stopping execution.

🎉 Project execution completed! Generated 1 tasks.